In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Import Library

In [ ]:
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2

import shutil
import itertools
import pathlib
from PIL import Image, ImageFile
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report

import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Conv2D, Flatten, Activation, Dense, Dropout, BatchNormalization, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import regularizers

### Data Preprocessing

In [ ]:
data_path = "/kaggle/input/brain-mri-images-for-brain-tumor-detection"
data = []
labels = []
categories = os.listdir(data_path)
for category in categories:
    category_path = os.path.join(data_path, category)
    for img_name in os.listdir(category_path):
        img_path = os.path.join(category_path, img_name)
        
        data.append(img_path)
        labels.append(category)

f_series = pd.Series(data, name = 'File_paths')
l_series = pd.Series(labels, name = 'Labels')
df = pd.concat([f_series, l_series], axis = 1)

In [ ]:
df

In [ ]:
train_df, test_df = train_test_split(df, train_size = 0.8, shuffle = True, random_state = 45, stratify = df['Labels'])

In [ ]:
img_size = (224, 224, 3)
batch_size = 16
TrainGen = ImageDataGenerator(
    rescale=1./255
)
TestGen = ImageDataGenerator(rescale = 1./255)

train_gen = TrainGen.flow_from_dataframe(train_df, 
                                        x_col = 'File_paths',
                                        y_col = 'Labels',
                                        target_size = (224, 224), 
                                        class_mode = 'categorical',
                                        color_mode = 'rgb',
                                        shuffle = True, 
                                        batch_size = batch_size)

test_gen = TestGen.flow_from_dataframe(test_df, 
                                        x_col = 'File_paths',
                                        y_col = 'Labels',
                                        target_size = (224, 224), 
                                        class_mode = 'categorical',
                                        color_mode = 'rgb',
                                        shuffle = False, 
                                        batch_size = batch_size)

In [ ]:
print(train_df['Labels'].value_counts())
print(test_df['Labels'].value_counts())

### Data Sample Display

In [ ]:
gen_dict = train_gen.class_indices
classes = list(gen_dict.keys())
images, labels = next(train_gen)

plt.figure(figsize = (15, 15))
for i in range(12):
    plt.subplot(4,4, i+1)
    image = images[i] 
    plt.imshow(image)
    index = np.argmax(labels[i])
    class_name = classes[index]
    plt.title(class_name, color = 'red', fontsize = 13)
    plt.axis('off')

plt.show()

### Model Structure

In [ ]:
image_shape = (224, 224, 3)
num_class = len(classes)

base_model = ResNet50(weights = 'imagenet', include_top = False, input_shape = image_shape, pooling = 'max')

for layer in base_model.layers[:-10]:
    layer.trainable = False


model = Sequential([
    base_model,
    BatchNormalization(),
    Dense(256, activation = 'relu'),
    Dropout(0.3),
    Dense(num_class, activation = 'softmax')
])

optimizer = Adam(learning_rate = 0.0001)

model.compile(
    optimizer = optimizer,
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)

early_stopping = EarlyStopping(monitor='val_loss', patience= 10, restore_best_weights=True)
model.summary()

In [ ]:
history = model.fit(
    train_gen,
    epochs = 30,
    batch_size = 8,
    verbose = 1,
    validation_data= test_gen, 
    validation_steps= None,
    callbacks=[early_stopping]
)

### Model Evaluation

In [ ]:
tr_acc = history.history['accuracy']
tr_loss = history.history['loss']
val_acc = history.history['val_accuracy']
val_loss = history.history['val_loss']
index_loss = np.argmin(val_loss)
val_lowest = val_loss[index_loss]
index_acc = np.argmax(val_acc)
acc_highest = val_acc[index_acc]
Epochs = [i+1 for i in range(len(tr_acc))]
loss_label = f'best epoch= {str(index_loss + 1)}'
acc_label = f'best epoch= {str(index_acc + 1)}'

# Plot training history
plt.figure(figsize= (20, 8))
plt.style.use('seaborn-v0_8-darkgrid')

plt.subplot(1, 2, 1)
plt.plot(Epochs, tr_loss, 'r', label= 'Training loss')
plt.plot(Epochs, val_loss, 'g', label= 'Validation loss')
plt.scatter(index_loss + 1, val_lowest, s= 150, c= 'blue', label= loss_label)
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(Epochs, tr_acc, 'r', label= 'Training Accuracy')
plt.plot(Epochs, val_acc, 'g', label= 'Validation Accuracy')
plt.scatter(index_acc + 1 , acc_highest, s= 150, c= 'blue', label= acc_label)
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout
plt.show()

In [ ]:
train_score = model.evaluate(train_gen, steps = 16, verbose = 1)
valid_score = model.evaluate(test_gen, steps = 16, verbose = 1)
test_score = model.evaluate(test_gen, steps = 16, verbose = 1)
print('Train Loss: ', train_score[0])
print('Train Accuracy: ', train_score[1])
print('-'*50)
print('Validation Loss: ', train_score[0])
print('Validation Accuracy: ', train_score[1])
print('-'*50)
print('Test Loss: ', train_score[0])
print('Test Accuracy: ', train_score[1])

In [ ]:
preds = model.predict(test_gen)
y_pred = np.argmax(preds, axis=1)
# Get class labels
g_dict = test_gen.class_indices
classes = list(g_dict.keys())

# Compute Confusion Matrix
cm = confusion_matrix(test_gen.classes, y_pred)
plt.figure(figsize=(10, 8))
plt.style.use('bmh')
sns.heatmap(cm, annot=True, fmt='d', cmap="YlGnBu", xticklabels=classes, yticklabels=classes, linewidths=0.5)
plt.title("Confusion Matrix", fontsize=14)
plt.xlabel("Predicted Label", fontsize=12)
plt.ylabel("True Label", fontsize=12)
plt.xticks(rotation=45)
plt.yticks(rotation=0)
plt.show()

In [ ]:
model.save('Brain_tumorM.h5')

In [ ]:
m = load_model('/kaggle/working/Brain_tumorM.h5')
loss, acc = m.evaluate(test_gen)
print(f"Model Accuracy: {acc:.4f}")

In [ ]:
m.summary()

In [ ]:
print(classification_report(test_gen.classes, y_pred, target_names = classes))

In [ ]:
def predict_img(image, model, class_indices):
    img = load_img(image, target_size = (224,224))
    img_array = img_to_array(img)
    img_array = img_array/255.0
    img_array = np.expand_dims(img_array, axis = 0)

    pred = model.predict(img_array)
    predicted_class_index = np.argmax(pred, axis=1)[0]      
    idx_to_label = {v: k for k, v in class_indices.items()}
    predicted_label = idx_to_label[predicted_class_index]

    return predicted_label


In [ ]:
img = '/kaggle/input/brain-mri-images-for-brain-tumor-detection/yes/Y116.JPG'
predict_img(img, model, train_gen.class_indices)